In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = 'QuadroP2000'

In [14]:
!ls ../../../prediction_model/data/raw_data/

1080ti	K40  k80  M60  P100  QuadroP2000  V100


In [12]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model/data/")

## Dense Layer

In [13]:
filename = os.path.join(DATA_DIR,'{}/benchmark_dense_{}_20190918.pkl'.format(model_name,model_name))
print(filename)
dfDense = pd.read_pickle(filename)

# for i in range(3,4):
#     dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20180925.pkl' %(file_name, i)))])

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/benchmark_dense_QuadroP2000_20190918.pkl


In [15]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [16]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [17]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,10000.000000,10000.000000,10000.000000,10000.0,10000.0,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,...,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,32.419500,2028.176300,2068.095100,0.0,32.0,2.853454,2.834536,2.814306,0.016782,1.359120e+08,...,0.081500,0.086200,0.080400,0.503600,0.082400,0.083800,0.252800,0.242900,0.252900,0.25140
std,18.422911,1181.425504,1180.505842,0.0,0.0,2.316097,2.308547,2.300937,0.014682,1.569006e+08,...,0.273615,0.280673,0.271925,0.500012,0.274987,0.277101,0.434639,0.428857,0.434696,0.43384
min,1.000000,1.000000,1.000000,0.0,32.0,0.690603,0.687408,0.658178,0.000103,1.995000e+03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,16.000000,994.000000,1049.500000,0.0,32.0,0.965786,0.955009,0.944185,0.007902,2.224653e+07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,32.000000,2020.000000,2054.000000,0.0,32.0,1.923490,1.912999,1.903987,0.013405,7.641068e+07,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,48.000000,3055.250000,3102.000000,0.0,32.0,4.142678,4.122782,4.088473,0.021862,1.918601e+08,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.00000
max,64.000000,4095.000000,4095.000000,0.0,32.0,13.935566,13.828993,13.771582,0.547301,9.950936e+08,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000


In [18]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [21]:
!ls /HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/benchmark_convolution_QuadroP2000_20190918.pkl

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/benchmark_convolution_QuadroP2000_20190918.pkl


In [22]:
file_name = os.path.join(DATA_DIR,'{}/benchmark_convolution_{}_20190918.pkl'.format(model_name,model_name))
print(file_name)
dfConv = pd.read_pickle(file_name)

header = dfConv.columns

# for i in range(2,3):
#     dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20180925.pkl' %(file_name, i)))])

/HDD2/ML/ml-performance-prediction/prediction_model/data/raw_data/QuadroP2000/benchmark_convolution_QuadroP2000_20190918.pkl


In [23]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [24]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [25]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,9383.000000,9383.000000,9383.000000,9383.0,9383.000000,9383.000000,9383.000000,9383.0,9383.000000,9383.000000,...,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000,9383.000000
mean,121.266652,14.333049,14.535863,0.0,3.978791,245.620271,0.505595,32.0,2.520729,82.986734,...,0.084301,0.078120,0.078973,0.523713,0.076095,0.083769,0.252691,0.253224,0.248961,0.245124
std,71.603786,65.871528,64.641642,0.0,2.013088,146.825455,0.499995,0.0,1.109047,152.331398,...,0.277854,0.268375,0.269710,0.499464,0.265164,0.277055,0.434578,0.434881,0.432434,0.430184
min,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.000000,32.0,1.000000,0.672007,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,59.000000,2.000000,2.000000,0.0,2.000000,117.000000,0.000000,32.0,2.000000,7.185888,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,117.000000,4.000000,4.000000,0.0,4.000000,242.000000,1.000000,32.0,3.000000,23.942184,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,184.000000,8.000000,8.000000,0.0,6.000000,372.000000,1.000000,32.0,4.000000,90.649009,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
max,250.000000,1852.000000,1942.000000,0.0,7.000000,512.000000,1.000000,32.0,4.000000,1677.131987,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [26]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))